In [4]:
!pip install --upgrade requests branca six jinja2 numpy chardet idna urllib3 certifi MarkupSafe
!conda install -c conda-forge folium --yes
!conda install -c conda-forge geopy --yes
!pip install bs4
!pip install uszipcode
!pip install tqdm

#import all the libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
import random
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import folium
#from folium.plugins import MiniMap
from geopy.geocoders import Nominatim
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn import metrics
from PIL import Image
import matplotlib.patches as mpatches
import types
from botocore.client import Config
import ibm_boto3
from bs4 import BeautifulSoup
from uszipcode import SearchEngine
import geopy
from geopy.extra.rate_limiter import RateLimiter
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

print('Libraries imported')

Requirement already up-to-date: requests in /opt/conda/envs/Python36/lib/python3.6/site-packages (2.24.0)
Requirement already up-to-date: branca in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.4.1)
Requirement already up-to-date: six in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.15.0)
Requirement already up-to-date: jinja2 in /opt/conda/envs/Python36/lib/python3.6/site-packages (2.11.2)
Requirement already up-to-date: numpy in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.19.1)
Requirement already up-to-date: chardet in /opt/conda/envs/Python36/lib/python3.6/site-packages (3.0.4)
Requirement already up-to-date: idna in /opt/conda/envs/Python36/lib/python3.6/site-packages (2.10)
Requirement already up-to-date: urllib3 in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.25.10)
Requirement already up-to-date: certifi in /opt/conda/envs/Python36/lib/python3.6/site-packages (2020.6.20)
Requirement already up-to-date: MarkupSafe in /opt/conda/envs/Pyt

In [5]:


#New York Geographical Data on Neighborhoods/Boroughs
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

#Load data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
#Take a look at the data
#newyork_data

#the relevant data is in the features key, which is basically a list of neighborhoods
NY_data = newyork_data['features']

# Transform the NY Data into a Pandas dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NYdf = pd.DataFrame(columns=column_names)

# Make sure the columns are correct
NYdf
NY_data[0]

for data in NY_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NYdf = NYdf.append({'Borough': borough,
                        'Neighborhood': neighborhood_name,
                        'Latitude': neighborhood_lat,
                        'Longitude': neighborhood_lon}, ignore_index=True)
    
    #Take only the neighborhoods from Manhattan
NYdf =NYdf[NYdf['Borough'] == 'Manhattan'].reset_index(drop=True)
NYdf.head(10)

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118
